# Time Series Analysis & Modeling
data used: [Chinese Monthly temperature data (1833-2013)](http://berkeleyearth.lbl.gov/auto/Regional/TAVG/Text/china-TAVG-Trend.txt)

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import seaborn as sns

## Table of Contents
 **1. Helpful links and my motivation**  
 **2. Why time series analysis**  
 **3. What kind of data is suitable for time series analysis **  
 **4. Math **  
 **5. Example**  

## Helpful Link & my motivation

If you already knew time series and want to learn how to apply those techniques in python, here is couple links that help me get started. 
* [youtube video on ARIMA](https://www.youtube.com/watch?v=tJ-O3hk1vRw)
* [A complete guide to time series analysis in python](https://www.analyticsvidhya.com/blog/2016/02/time-series-forecasting-codes-python/)
* [A complete guide to time series analysis in python 1(nicer formatting)](https://www.digitalocean.com/community/tutorials/a-guide-to-time-series-forecasting-with-arima-in-python-3)
* [A complete guide to time series analysis in python 2(nicer formatting)](http://www.seanabu.com/2016/03/22/time-series-seasonal-ARIMA-model-in-python/)

Those blog already did great jobs on presenting complete analysis on complicate time series. The main reason I started thinking about writing a blog about time series is that recently I had a EPIC FAIL during one of technical interviews.

<img src="https://media.giphy.com/media/KGTTNpVuGVhN6/giphy.gif">

My motivations for this blog are:
* Refresh my memory on time series
* Present models step by step from the most rudimentary models to complicate ones



## Why do we use time series analysis?

The most important characteristic of time series analysis, in my opinion, is the order of the data. Order matters. If a series of temporal data is given - *1,2,3*, I will ask the question what is the relationship between 1 at $t_0$ and 3 at $t_h$, where h can be 0,1,...n.

Through exploring the temporal relationship among data, we turn a dimensionless problem to a dimentional problem. How? 

* Assume data 1,2,3 didn't happen in a time order. How do we estimate the next most likely happened number? Mean, median, mode, and probably construct probability distribution. But we can't run a regression method (require at least 1 dimension feature space).  
* However, if the data is temporal and they are collected at $t_0$, $t_1$,... and $t_n$. We can make hypothesis about value at $t_i$ is linearly correlated with value at $t_{i-2}$. Putting that hypothesis down as math equation, it would be $$x_t =\epsilon_t +\alpha*x_{t-2}$$ and now the problem turns into a 1-d regression problem
* If you agree with the logic above - the problem is a 1d problem, can you picture it being a p-dimention problem ($p<n$)? $$x_t = \epsilon_t+\sum^p_1{\alpha_i*x_{t-i}}$$ This is a type of time series model called autoregressive model, **AR(p)**, and we will cover it later. 

The two euqations above share an important trait - **we are using data from the PAST to forecast/estimate the FUTURE.** 

From [this blog post](http://userwww.sfsu.edu/efc/classes/biol710/timeseries/timeseries1.htm), we see other goals we can achieve by analyzing time series
<img src="image/goal_ts.png" alt="Goal of TS analysis" style="width: 80%"/>


## What kind of data is suitable for time series analysis

We are trying to extract meaningful statistic out of temporal data. However, if a temporal data is completely random - no systematic pattern in relation to time, then we cannot find insight. 

### what kind of data do we use here

**The time series we are concerned will have patterns and noises.** We are going to damp the noises and recognize the patterns. The data we are concerned are all [stationary](https://en.wikipedia.org/wiki/Stationary_process) (weakly stationary).

*If you understand what stationary means, keep reading; otherwise, you can skip this!*
* The assumption behind stationary process is very strong: it requires identical joint probaility distributions when shifted in time. In practice, it's hard to meet such requirement.
* So someone invented the notion of weakly stationary: the time series $x_i, i =1,2,3,...t$ has to be
    * constant mean and variance (finite) across time 
    * constant autocorrvariance s.t. $cov(x_1,x_2) = cov(x_{1+h},x_{2+h})$  

